In [173]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from scipy.stats import f_oneway
from sklearn.linear_model import LinearRegression

In [131]:
mc_df=pd.read_csv('Mercedes (1).csv')
mc_df.shape

(4209, 378)

In [132]:
init_cat_cols=[col for col in mc_df.columns if mc_df[col].dtype=='object']## Finding catg cols

In [135]:
init_con_cols=[col for col in mc_df.columns if mc_df[col].dtype=='int64' or mc_df[col].dtype=='float64']
## Initial Data set cont columns

In [136]:
# Droping cont cols becoz of only single variable that too these are catg actually
cols_drop_uniq_val=[]
for col in con_cols:
    if len(mc_df[col].value_counts()) ==1 or len(mc_df[col].value_counts())==mc_df.shape[0]:
        cols_drop_uniq_val.append(col)
cols_drop_uniq_val

['ID',
 'X11',
 'X93',
 'X107',
 'X233',
 'X235',
 'X268',
 'X289',
 'X290',
 'X293',
 'X297',
 'X330',
 'X347']

In [137]:
#droping single value catg columns in datas et which is no use for model feeding
mc_df.drop(columns=cols_drop_uniq_val,inplace=True)

In [143]:
con_cols=[col for col in mc_df.columns if mc_df[col].dtype=='int64' or mc_df[col].dtype=='float64']
## Initial Data set cont columns
con_cols

['y']

In [148]:
cat_cols=[col for col in mc_df.columns if mc_df[col].dtype=='object']## Finding catg cols

In [142]:
# converting all cont cols to catg cols
#mc_df[con_cols]=mc_df[con_cols].astype(object)
for col in con_cols:
    if len(mc_df[col].value_counts().index) == 2:
        mc_df[col]=mc_df[col].astype(object)
    else:
        mc_df[col]=mc_df[col].astype(float)

In [144]:
mc_df['y']

0       130.81
1        88.53
2        76.26
3        80.62
4        78.02
         ...  
4204    107.39
4205    108.77
4206    109.22
4207     87.48
4208    110.85
Name: y, Length: 4209, dtype: float64

In [145]:
len(mc_df['X15'])==mc_df.shape[0]

True

In [146]:
y=mc_df['y']

In [149]:
#performing anova to find out corelated columns
def anova(col):
    catg=mc_df[col].value_counts().index
    res=f_oneway(*[mc_df[mc_df[col]==cat]['y'] for cat in catg])
    return res[1],col


cont_data_all_cols=[anova(col) for col in cat_cols]
#cont_data_cor_cols
anova_df=pd.DataFrame(cont_data_all_cols,columns =['value', 'col_name'])
anova_df.head(20)

,value,col_name
0,0.000000e+00,X0
1,1.128032e-24,X1
2,1.930684e-196,X2
3,1.251233e-36,X3
4,4.920920e-02,X4
5,4.035847e-04,X5
6,3.615937e-06,X6
7,1.269254e-14,X8
8,8.003760e-02,X10
9,5.364599e-09,X12


In [179]:
corr_cont_cols=anova_df[anova_df['value'] < 0.05]
corr_cont_col_names=list(corr_cont_cols['col_name'])
len(corr_cont_col_names)

250

In [151]:
mainfo_df=mc_df.copy()## Creating copy of DF for deleting unwanted columns
mainfo_df.drop(columns=corr_cont_col_names,inplace=True)## Now this DF contains unwanted columns so that it can pass to next step
ls_cols_remove=list(mainfo_df.columns)

In [152]:
mc_df.drop(columns=ls_cols_remove,inplace=True) ## Unwanted cols deleting and having only correlated cols
mc_df## only Correlated cols in main DF

,X0,X1,X2,X3,X4,X5,X6,X8,X12,X13,...,X371,X372,X373,X376,X377,X378,X379,X380,X382,X383
0,k,v,at,a,d,u,j,o,0,1,...,0,0,0,0,1,0,0,0,0,0
1,k,t,av,e,d,y,l,o,0,0,...,0,0,0,0,0,0,0,0,0,0
2,az,w,n,c,d,x,j,x,0,0,...,0,0,0,0,0,0,0,0,1,0
3,az,t,n,f,d,x,l,e,0,0,...,0,1,0,0,0,0,0,0,0,0
4,az,v,n,f,d,h,d,n,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,ak,s,as,c,d,aa,d,q,0,0,...,0,0,0,0,0,0,0,0,0,0
4205,j,o,t,d,d,aa,h,h,0,0,...,0,0,0,1,0,0,0,0,0,0
4206,ak,v,r,a,d,aa,g,e,1,1,...,0,0,0,0,1,0,0,0,0,0
4207,al,r,e,f,d,aa,l,u,0,0,...,0,0,0,0,0,0,0,0,0,0


In [206]:
x_train,x_test,y_train,y_test=train_test_split(mc_df,y,random_state=49,test_size=0.2)
x_train.shape

(3367, 250)

In [207]:
[col for col in mc_df.columns if mc_df[col].dtype=='int64' or mc_df[col].dtype=='float64']
#Validating no cont cols in Data set

[]

In [208]:
ful_cat_cols=[col for col in mc_df.columns if mc_df[col].dtype=='object']
len(ful_cat_cols)
#Validating all  cls in Data set are catg

250

In [209]:
x_train[ful_cat_cols].isnull().sum().any()## no missing values

False

In [210]:
cat_encd_train= pd.get_dummies(x_train[ful_cat_cols])
cat_encd_test=pd.get_dummies(x_test[ful_cat_cols])

In [211]:
cat_encd_train.shape,cat_encd_test.shape

((3367, 674), (842, 651))

In [212]:
cat_encd_train_final,cat_encd_test_final=cat_encd_train.align(cat_encd_test,join='inner',axis=1)
cat_encd_train_final.shape###aligning train & test data one hot encoded catg columns due to unqual no of columns i.e no of cilumns would differ for that we align to get same

(3367, 646)

In [213]:
dtree_reg=DecisionTreeRegressor()

In [214]:
dtree_reg.fit(cat_encd_train_final,y_train)

DecisionTreeRegressor()

In [215]:
y_test_pred=dtree_reg.predict(cat_encd_test_final)
y_test_pred

array([106.52      , 109.21      ,  98.72      ,  98.765     ,
        97.76      ,  90.11      , 112.8       ,  99.99      ,
        75.05      ,  98.95      ,  97.55      , 108.3       ,
       108.99      ,  92.65833333,  76.64      , 114.88      ,
       113.31      ,  86.51      ,  98.61      , 106.67      ,
        89.81      ,  93.12      ,  87.81      , 110.32      ,
        98.88      ,  89.79      , 113.86      ,  91.62      ,
       105.89      ,  89.46      ,  95.15      ,  93.45      ,
        87.3       , 111.345     , 120.38      ,  78.225     ,
        88.41      , 114.32      , 112.32      ,  87.05      ,
        83.96      , 100.05      ,  89.97      ,  94.31      ,
        92.54      , 116.2       ,  89.69      ,  93.27      ,
        94.5       ,  93.295     ,  92.08      , 108.83      ,
       103.06      ,  87.51      , 100.68      ,  90.04      ,
        90.52      , 117.98      ,  96.41      , 103.96      ,
       108.8       , 100.05      , 111.44      ,  93.97

In [225]:
dtree_reg.score(cat_encd_test_final,y_test)

-0.041340950157426004

In [226]:
lin_reg=LinearRegression()
lin_reg.fit(cat_encd_train_final,y_train)

LinearRegression()

In [227]:
y_test_pred_lnr=lin_reg.predict(cat_encd_test_final)

In [228]:
lin_reg.score(cat_encd_test_final,y_test)

-2.0380099741629735e+21